In [ ]:
import numpy as np
import pandas as pd

import librosa
import pywt
from skimage.restoration import denoise_wavelet
from loess.loess_1d import loess_1d
import statsmodels.api as sm
import scipy

import matplotlib.pyplot as plt
from IPython.display import Audio
from tqdm import tqdm

import argparse
from os import listdir
from os.path import join

In [ ]:

def read_audio_filenames(directory: str) -> list:
    '''

    '''
    return [ filename for filename in listdir(directory) if filename.endswith('.wav') ]


def create_icbhi2017_audio_dataframe(filenames: list, basedir: str) -> pd.DataFrame:
    '''
    '''
    rows = []

    for filename in tqdm(filenames):

        parts = filename.split('.')[0].split('_')
        id = parts[0]
        audio_content = read_audio_content(join(basedir, filename))
        segments = segment_audio_content(audio_content)

        for segment in segments:

            # wavelet denoise
            segment = wavelet_denoise(segment)

            # loess
            segment = apply_loess(segment)

            # z-score normalization
            segment = zscore_normalize(segment)

            rows.append([id, segment])

    df = pd.DataFrame(rows, columns=['ID', 'audio_data'])
    return df


def read_audio_content(filepath: str, sample_rate=4000):
    '''
    '''
    data, sr = librosa.load(filepath, sr=sample_rate)
    return data, sr


def segment_audio_content(audio, sample_rate=4000, segment_length=5):
    ''''''
    segments = []
    audio_length = len(audio) / sample_rate

    # print('Audio length', audio_length)

    segment_index = 0
    while segment_index < audio_length:
        t_start = segment_index * sample_rate
        t_end = t_start + segment_length * sample_rate

        segment = audio[ t_start : t_end ]
        segments.append(np.pad(segment, (0, sample_rate * segment_length - len(segment)), 'constant'))
        segment_index += segment_length

    # print(segment_index / segment_length, 'segments read')
    # print([ len(s) for s in segments ])
    return segments


def preprocess(audio):
    ''''''
    processed = wavelet_denoise(audio)
    processed = apply_loess(processed)
    processed = zscore_normalize(processed)
    return processed


def wavelet_denoise(audio):
    '''Wavelet denoise
    '''
    return denoise_wavelet(audio, wavelet='db5', method='BayesShrink', mode='soft', wavelet_levels=4)
    # return pywt.wavedec(audio, 'db5', mode='zero', level=4)[0]


def apply_loess(audio, frac=0.1):
    '''
    '''
    lowess = sm.nonparametric.lowess
    return audio - lowess(audio, np.arange(0, len(audio), 1), frac=frac)
    

def zscore_normalize(audio):
    '''
    '''
    return scipy.stats.zscore(audio)



In [ ]:
LABELS_OF_INTEREST = ['Healthy', 'Asthma', 'Pneumonia', 'Bron', 'COPD', 'Heart failure']

ICBHI_DATASET_BASEDIR = 'Respiratory_Sound_Database'
ICBHI_AUDIO_TXT_DIR = join('..', 'dados', ICBHI_DATASET_BASEDIR, 'audio_and_txt_files')

SEGMENTS_LENGTH = 5

In [ ]:
KING_ABDULAH_DATASET_BASEDIR = join('Audio_Files')

king_abdulah_audio_filenames = read_audio_filenames(KING_ABDULAH_DATASET_BASEDIR)


labels_map = {
    'N': 'Healthy',
    'Copd': 'COPD',
}

audios = []
for filename in tqdm(king_abdulah_audio_filenames):

    (id, diagnosis) = filename.split(',')[0].split('_')
    diagnosis = diagnosis.capitalize()
    audio, sample_rate = read_audio_content(join(KING_ABDULAH_DATASET_BASEDIR, filename))
    segments = segment_audio_content(audio, sample_rate, SEGMENTS_LENGTH)

    for (seg_no, segment) in enumerate(segments):

        label = labels_map.get(diagnosis, diagnosis)

        processed_segment = preprocess(segment)

        row = {'ID': id, 'segment_no': seg_no, 'audio_data': processed_segment, 'Diagnosis': label}
        audios.append(row)

df_audio_diagnosis_l = pd.DataFrame(audios)
df_audio_diagnosis_l = df_audio_diagnosis_l[df_audio_diagnosis_l.Diagnosis.isin(LABELS_OF_INTEREST)]
df_audio_diagnosis_l.groupby('Diagnosis')['Diagnosis'].count()